# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [59]:
#https://arlweb.msha.gov/drs/drshome.htm

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [60]:
#//*[@id="inputdrs"]

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [61]:
#The buttons xpath is:
#//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input

### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [62]:
from selenium import webdriver

In [63]:
driver = webdriver.Chrome()

In [64]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [65]:
input_field = driver.find_element_by_name('MineId')

In [66]:
input_field.send_keys('3503598')

In [67]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

In [68]:
#Versuch via BeautifulSoup

In [69]:
#from bs4 import BeautifulSoup

In [70]:
#doc = BeautifulSoup(driver.page_source, 'html.parser')

In [71]:
#doc.find_all('tr')

In [72]:
name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')

In [73]:
name_tag.text

'Newberg Rock & Dirt'

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [74]:
import pandas as pd
df = pd.read_csv("mines-subset.csv", dtype={'id': 'str'}) # sagt Pandas: behandle Zahl als String
df.head()

,id
0,4104757
1,0801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

In [75]:
# original number has a '0'

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [76]:
#it looks okay

### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [81]:
def scrape_name(row):   
    driver.get('https://arlweb.msha.gov/drs/drshome.htm')
    input_field = driver.find_element_by_name('MineId')
    input_field.send_keys(row['id'])
    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click() 
    name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
    return name_tag.text # wie ein dictionary behandeln

df['name'] = df.apply(scrape_name, axis=1) #ist ein for loop für pandas

In [82]:
df.head()

,id,name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*